# Operation Research Fall 99 <img src = 'https://ece.ut.ac.ir/cict-theme/images/footer-logo.png' alt="Tehran-University-Logo" width="150" height="150" align="right">
## Final Project
### Dr. Mohammad Shokri
### By Omid Vaheb, Amirhossein Nazer, and Somayeh Gashtasbi

First of al, we import required libraries which is only MIP. Python MIP is a collection of Python tools for the modeling and solution of Mixed-Integer Linear programs (MIPs). MIP syntax was inspired by Pulp.

In [89]:
import mip
from mip import Model, xsum, minimize, BINARY

Then we define 2 dictionaries: d for cities or demands and M for power plants or sources.

In [90]:
d = {4:1710, 6:467, 10:383, 11:226, 14:1129, 15:2917, 16:411, 18:281, 20:949, 21:325}
M = {1:456, 2:460, 3:3055, 5:223, 7:334, 8:4000, 9:203, 12:250, 13:3271, 17:1400, 19:1457, 22:415, 23:330}

We also define 2 lists showing indexes of these cities and power plants.

In [91]:
I = [4, 6, 10, 11, 14, 15, 16, 18, 20, 21]
J = [1, 2, 3, 5, 7, 8, 9, 12, 13, 17, 19, 22, 23]

Then we define a dictionary called c which consists of lenghts between nodes of our graph, cities and power plants.

In [92]:
c = {(1,4):842, (2,4):886, (3,4):74, (4,6):282, (5,6):450, (6,10):94, (7,10):220, (10,19):400, (10,14):124, (8,11):546,
     (9,11):52, (11,12):545, (11,13):95, (11,14):95, (11,15):125, (13,15):119, (14,15):63, (10,15):100, (15,16):90,
     (10,18):312, (16,18):210, (16,17):96, (17,18):215, (3,21):165, (4,21):214, (4,18):144, (21,22):1015, (20,21):210,
     (18,21):235, (18,20):356, (20,23):878}

The next step is to define number of bundles for each power transmission line.

In [93]:
B = {(1,4):2, (2,4):2, (3,4):2, (4,6):2, (5,6):2, (6,10):1, (7,10):2, (10,19):2, (10,14):1, (8,11):2, (9,11):1, (11,12):2,
     (11,13):2, (11,14):1, (11,15):2, (13,15):2, (14,15):2, (10,15):1, (15,16):2, (10,18):2, (16,18):2, (16,17):2,
     (17,18):2, (3,21):2, (4,21):2, (4,18):2, (21,22):2, (20,21):1, (18,21):2, (18,20):2, (20,23):2}

Now we add variables which are the power transmiting between lines. First we add the lines between power plants and cities.

In [94]:
model = mip.Model()
x = {}
for i in I:
    for j in J:
        if ((i,j) in c or (j,i) in c) and not ((i,j) in x or (j,i) in x):
            if i > j:
                x[j,i] = model.add_var(var_type = mip.INTEGER)
            else:
                x[i,j] = model.add_var(var_type = mip.INTEGER)

Now is the time to add the lines between cities.

In [95]:
for i in I:
    for j in I:
        if ((i,j) in c or (j,i) in c) and not ((i,j) in x or (j,i) in x):
                x[j,i] = model.add_var(var_type = mip.INTEGER)
                x[i,j] = model.add_var(var_type = mip.INTEGER)

Now we add the constraints regarding this problem. First and foremost, we write the constraint for each city in which sum of edges going to city minus sum of edges coming from it, is equal or higher than demand of that city.

In [96]:
for i in I:
    constraints = []
    constraints2 = []
    for j in I + J:
        if (i,j) in x and (j,i) in x:
            if i < j:
                constraints.append(x[i,j])
                constraints2.append(x[j,i])
            else:
                constraints.append(x[i,j])
                constraints2.append(x[j,i])
        elif (i,j) in x:
            constraints.append(x[i,j])
        elif (j,i) in x:
            constraints.append(x[j,i])
    model.add_constr((xsum(constraints) - xsum(constraints2)) >= d[i])

4,6
-6,4
4,18
-18,4
4,21
-21,4
1,4
2,4
3,4
**********
6,4
-4,6
6,10
-10,6
5,6
**********
10,6
-6,10
10,14
-14,10
10,15
-15,10
10,18
-18,10
7,10
10,19
**********
11,14
-14,11
11,15
-15,11
8,11
9,11
11,12
11,13
**********
14,10
-10,14
14,11
-11,14
14,15
-15,14
**********
15,10
-10,15
15,11
-11,15
15,14
-14,15
15,16
-16,15
13,15
**********
16,15
-15,16
16,18
-18,16
16,17
**********
18,4
-4,18
18,10
-10,18
18,16
-16,18
18,20
-20,18
18,21
-21,18
17,18
**********
20,18
-18,20
20,21
-21,20
20,23
**********
21,4
-4,21
21,18
-18,21
21,20
-20,21
3,21
21,22
**********


Then we add the constraint regarding each power plant. Sum of edges going out from power plant is equal or less than the power it produces.

In [97]:
for j in J:
    constraints = []
    for i in I:
        if (i,j) in x:
            constraints.append(x[i,j])
        elif (j,i) in x:
            constraints.append(x[j,i])
    model.add_constr(xsum(constraints) <= M[j])

Final constraint is on power lines. Each power line cannot transmit any power hiher than 1000MW because of physical restrictions.

In [98]:
for (i,j) in x:
    model.add_constr(x[i,j] <= 1000)

The final step is to define the cost function which is multiplication of power of each line in the lenght of it divided by number of bundles.

In [99]:
costs = []
for (i,j) in x:
    if i < j:
        costs.append(c[i,j] * x[i,j] / B[i,j])
    else:
        costs.append(c[j,i] * x[i,j] / B[j,i])

Finally we optimize the cost function and check if our solution satisfies out constraints. In this case solution satisfies all constraints and you can check it in solution.jpg file.

In [100]:
model.objective = mip.minimize(sum(costs))
print(model.optimize())
print("Minimum cost is " + str(model.objective_value) + ".")

OptimizationStatus.OPTIMAL
Minimum cost is 1604267.5.


We can check the value of ach variable in the optimum point.

In [101]:
for (i,j) in x:
    print("The power transmiting from " + str(i) + " to " + str(j) + " : " +  str(x[i,j].x))

The power transmiting from 1 to 4 : 456.0
The power transmiting from 2 to 4 : 0.0
The power transmiting from 3 to 4 : 1000.0
The power transmiting from 5 to 6 : 223.0
The power transmiting from 7 to 10 : 334.0
The power transmiting from 10 to 19 : 1000.0
The power transmiting from 8 to 11 : 602.0
The power transmiting from 9 to 11 : 203.0
The power transmiting from 11 to 12 : 250.0
The power transmiting from 11 to 13 : 1000.0
The power transmiting from 13 to 15 : 1000.0
The power transmiting from 16 to 17 : 921.0
The power transmiting from 17 to 18 : 479.0
The power transmiting from 20 to 23 : 330.0
The power transmiting from 3 to 21 : 1000.0
The power transmiting from 21 to 22 : 0.0
The power transmiting from 6 to 4 : 0.0
The power transmiting from 4 to 6 : 0.0
The power transmiting from 18 to 4 : 0.0
The power transmiting from 4 to 18 : 198.0
The power transmiting from 21 to 4 : 0.0
The power transmiting from 4 to 21 : 56.0
The power transmiting from 10 to 6 : 0.0
The power transmiti